In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

import glob
import h5py

import obspy
from obspy.signal.rotate import rotate_ne_rt
from obspy.clients.fdsn import Client as FDSN_Client
from obspy.geodetics import base
from obspy.taup import TauPyModel

import pyproj

from tqdm.notebook import tqdm

%load_ext autoreload
%autoreload 2

# Simulate strain rates

In [ ]:
auspass = FDSN_Client('http://auspass.edu.au:8080')
inv = auspass.get_stations(network="2B", level='response')
model = TauPyModel(model="iasp91")

locations = {}
for station in inv[0]:
    locations[station.code] = (station.latitude, station.longitude)

station_pairs = []
for i, station in enumerate(inv[0]):
    if station.code in ['SIS01', 'SIS05']:
        continue
    for j in range(i+1, len(inv[0])):
        d = base.gps2dist_azimuth(inv[0][i].latitude, inv[0][i].longitude, 
                                  inv[0][j].latitude, inv[0][j].longitude)[0]
        if d <= 150:
            print('Distance between {} and {}: {:.4f} meters'.format(inv[0][i].code, inv[0][j].code, d))
            station_pairs.append((inv[0][i].code, inv[0][j].code))

In [ ]:
path = './data/waveforms/'
files = glob.glob(path + '*.mseed')

n_train = int(len(files) * 0.8)
n_test = len(files) - n_train

geodesic = pyproj.Geod(ellps='WGS84')

In [ ]:
train_sr = []
for i in tqdm(range(n_train)):
    st = obspy.read(files[i])

    for j, (station1, station2) in enumerate(station_pairs):
        
        lat1 = locations[station1][0]
        lon1 = locations[station1][1]

        idx1 = list(locations.keys()).index(station1)
        north1 = st[3*idx1].data[:9000]    # station 1 north component
        east1 = st[3*idx1+1].data[:9000]   # station 1 east component

        lat2 = locations[station2][0]
        lon2 = locations[station2][1]
        idx2 = list(locations.keys()).index(station2)
        north2 = st[3*idx2].data[:9000]    # station 2 north component
        east2 = st[3*idx2+1].data[:9000]   # station 2 east component

        # Calculate the back azimuth and distance between the two stations
        _, back_azimuth, distance_in_m = geodesic.inv(lon1, lat1, lon2, lat2)

        # Rotate the waveforms to the radial component
        radial1, _ = rotate_ne_rt(north1, east1, back_azimuth % 360)
        radial2, _ = rotate_ne_rt(north2, east2, back_azimuth % 360)

        train_sr.append((radial1 - radial2) / distance_in_m)

if not os.path.exists('./data/preprocessed/'):
     os.makedirs('./data/preprocessed/')
np.save('./data/preprocessed/SIS-rotated_train_50Hz.npy', np.stack(train_sr))

In [ ]:
test_sr = []
for i in tqdm(range(n_train, len(files))):
    st = obspy.read(files[i])

    for j, (station1, station2) in enumerate(station_pairs):
        
        lat1 = locations[station1][0]
        lon1 = locations[station1][1]

        idx1 = list(locations.keys()).index(station1)
        north1 = st[3*idx1].data[:9000]    # station 1 north component
        east1 = st[3*idx1+1].data[:9000]   # station 1 east component

        lat2 = locations[station2][0]
        lon2 = locations[station2][1]
        idx2 = list(locations.keys()).index(station2)
        north2 = st[3*idx2].data[:9000]    # station 2 north component
        east2 = st[3*idx2+1].data[:9000]   # station 2 east component

        # Calculate the back azimuth and distance between the two stations
        _, back_azimuth, distance_in_m = geodesic.inv(lon1, lat1, lon2, lat2)

        # Rotate the waveforms to the radial component
        radial1, _ = rotate_ne_rt(north1, east1, back_azimuth % 360)
        radial2, _ = rotate_ne_rt(north2, east2, back_azimuth % 360)

        test_sr.append((radial1 - radial2) / distance_in_m)

if not os.path.exists('./data/preprocessed/'):
     os.makedirs('./data/preprocessed/')
np.save('./data/preprocessed/SIS-rotated_test_50Hz.npy', np.stack(test_sr))

In [ ]:
sr = np.stack(test_sr)

plt.figure(figsize=(7, 15))

for i in range(len(sr[:100])):
    x = sr[i] / sr[i].std()
    plt.plot(x + 5*i, c="k", lw=0.5, alpha=1)

plt.tight_layout()
plt.show()

# Extract traffic signals

In [ ]:
dx = 4    # channel spacing
fs = 50   # sampling frequency

def speed2slowness(speed):
    speed = speed * 1000 / 3600
    return 1/speed

def slowness2speed(slowness):
    speed = 1 / slowness
    return speed * 3600 / 1000

shift2slowness = lambda shift: shift / (dx * fs) # s/m
shift2speed = lambda shift: (60 * 60 * dx * fs) / (shift * 1000) # km/h

dir1 = './data/index0185_0205/'
dir2 = './data/index0275_0295/'
files1 = [f'{m//60:02d}{m%60:02d}01' for m in range(7*60+55, 8*60+30)]
files2 = [f'{m//60:02d}{m%60:02d}01' for m in range(9*60+30, 10*60)]

prefix = 'south30_50Hz_UTC_20230401_'
suffix = '.001.h5'

def get_chunk(idx, d, f):
    file = d + prefix + f[idx] + suffix
    data = (h5py.File(file,'r')['DAS'][:].T)[81:4978]
    file = d + prefix + f[idx+1] + suffix
    data = np.hstack([data, (h5py.File(file,'r')['DAS'][:].T)[81:4978]])
    file = d + prefix + f[idx+2] + suffix
    data = np.hstack([data, (h5py.File(file,'r')['DAS'][:].T)[81:4978]])
    return data

In [ ]:
# manually selected picks
train_picks = [(2,190,3200,-0.034), (3,440,2000,-0.035), (4,870,2000,-0.033), (5,1230,1500,-0.03), (7,2230,2000,-0.039),
               (15,3050,7500,0.045), 
               (21,1320,7500,0.048), (23,750,7000,0.05),
               (23,1880,7500,0.057), (25,1285,7500,0.051), (27,740,7500,0.054), (29,220,7000,0.052),
              ]
test_picks = [(1,625,1500,-0.047), (2,935,1500,-0.04), (3,1290,1500,-0.037), (5,2070,1500,-0.048), 
              (5,4000,6500,0.042), (10,2440,7000,0.048), (12,1720,7000,0.040), (13,1270,7500,0.040), 
             ]

if not os.path.exists('./data/preprocessed/'):
     os.makedirs('./data/preprocessed/')

for n, p in enumerate(tqdm(train_picks)):
    chunk_idx, i, j, slowness = p
    shift = dx * fs * slowness
    data = get_chunk(chunk_idx, dir1, files1)

    strain_rates = []
    for x in range(512):
        jj = j - int(x*shift)
        sr = data[i+x, jj-30*fs:jj+30*fs]
        strain_rates.append(sr)
    strain_rates = np.stack(strain_rates)

    direction = 'inc' if slowness < 0 else 'dec'
    np.save('./data/preprocessed/traffic_train_50Hz_{:02d}_{}.npy'.format(n, direction), strain_rates)

for n, p in enumerate(tqdm(test_picks)):
    chunk_idx, i, j, slowness = p
    shift = dx * fs * slowness
    data = get_chunk(chunk_idx, dir1, files1)

    strain_rates = []
    for x in range(512):
        jj = j - int(x*shift)
        sr = data[i+x, jj-30*fs:jj+30*fs]
        strain_rates.append(sr)
    strain_rates = np.stack(strain_rates)

    direction = 'inc' if slowness < 0 else 'dec'
    np.save('./data/preprocessed/traffic_test_50Hz_{:02d}_{}.npy'.format(n, direction), strain_rates)

In [ ]:
plt.imshow(strain_rates / strain_rates.std(axis=-1, keepdims=True), origin='lower',
           interpolation='none', cmap='seismic', aspect='auto', vmin=-1, vmax=1)
plt.show()

# Extract real DAS samples

general performance for a large earthquake at similar distance, with and without traffic
- 2023p202703, M4.7, heavy traffic, distance = 175 km

general performance for small earthquakes at increasing distances (with no traffic)
- 2023p215452, M2.7, no traffic, distance = 76 km

general performance for very small earthquakes very close to the fibre (less than array aperture - 30 km)
- 2023p273854, M1.9, minimal traffic, distance = 10 km

general performance for small earthquakes at increasing distances (with some traffic)
- 2023p181489, M3.3, some traffic, distance = 92 km
- 2023p197461, M3.6, some traffic, distance = 159 km
- 2023p215753, M3.1, some traffic, distance = 191 km

In [ ]:
path = './data/'
train_picks = ['2023p214660', '2023p300579', '2023p303853', '2023p218553', '2023p326201', '2023p313442', 
               '2023p265881', '2023p171245', '2023p291534', '2023p293493', '2023p278798', '2023p321451', 
               '2023p166738', '2023p307984', '2023p273765', '2023p209788', '2023p326200', '2023p299899', 
               '2023p212693', '2023p303138', '2023p252684', '2023p167136', '2023p202039', '2023p235224', 
               '2023p272380', '2023p172434', '2023p152354', '2023p316959', '2023p178129']
test_picks = ['2023p202703', '2023p215452', '2023p273854', '2023p181489', '2023p197461', '2023p215753']

os.makedirs('./data/preprocessed/real_train/', exist_ok=True)
os.makedirs('./data/preprocessed/real_test/', exist_ok=True)

In [ ]:
for pick in train_picks:

    data = np.hstack([h5py.File(path + pick + '/' + p)['DAS'][:].T for p in sorted([p for p in os.listdir(path + pick + '/')])])
    
    fig, ax = plt.subplots(1, 1, sharex=True, figsize=(12,6))
    ax.imshow(data, origin='lower', interpolation='nearest', cmap='seismic', aspect='auto', vmin=-100, vmax=100)
    plt.tight_layout()
    plt.show()
    
    with h5py.File(path + 'preprocessed/real_train/' + pick + '.h5', 'w') as hf:
        hf.create_dataset('DAS', data=data)

In [ ]:
for pick in test_picks:

    data = np.hstack([h5py.File(path + pick + '/' + p)['DAS'][:].T for p in sorted([p for p in os.listdir(path + pick + '/')])])
    
    fig, ax = plt.subplots(1, 1, sharex=True, figsize=(12,6))
    ax.imshow(data, origin='lower', interpolation='nearest', cmap='seismic', aspect='auto', vmin=-100, vmax=100)
    plt.tight_layout()
    plt.show()
    
    with h5py.File(path + 'preprocessed/real_test/' + pick + '.h5', 'w') as hf:
        hf.create_dataset('DAS', data=data)